# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

*course assignment instructions in italics*

## Part 1 - Scraping Data from Wikipedia Pages

for the scraping I used a tutorial from [simpleanalytical.com](https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas)

##### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

*For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.*

*Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe:*

In [4]:
# import the library we use to open URLs
import urllib.request

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [6]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [7]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [194]:
# let's look at the html code
#print(soup.prettify()) # <- uncomment to have a look at it

There we have it. Our table starts with **`<table class="wikitable">`**

let's fetch the table...

In [195]:
# fetch our table
wikiTable=soup.find('table', class_='wikitable')
#wikiTable # <-uncomment to have a look at it

In [31]:
# import pandas to be able to work with dataframes
import pandas as pd

- *The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood*

In [148]:
# define the dataframe columns
columnNames = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
dfPC = pd.DataFrame(columns=columnNames)

- *Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*
- *If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.*
- *More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will - notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.*

In [185]:
# fill wiki table in a dataframe
A=[]
B=[]
C=[]
    
for row in wikiTable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        tmpPostalCode=cells[0].find(text=True).rstrip()
        tmpBorough=cells[1].find(text=True).rstrip()
        tmpNeighborhood=cells[2].find(text=True).rstrip()
        tmpNeighborhood=tmpNeighborhood.replace(' /',',')   # two rows will be combined into 
                                                            # one row with the neighborhoods separated with a comma
        if tmpBorough!='Not assigned': # Ignore cells with a borough that is Not assigned
            A.append(tmpPostalCode)
            B.append(tmpBorough)
            if tmpNeighborhood=='': # cell has a borough but a Not assigned neighborhood
                C.append(tmpBorough) # neighborhood will be the same as the borough
            else:
                C.append(tmpNeighborhood)

so now let's fill the contents of the list into a dataframe

In [191]:
df=pd.DataFrame({columnNames[0]: A, columnNames[1]: B, columnNames[2]: C})
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


- *In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.*

In [192]:
df.shape

(103, 3)

**So we have a dataframe with 103 unique rows and 3 columns.**

## Part 2 - Making Use of geospatial Data

*Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.*

*In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.*

*The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:*

```python
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]```

*Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data*

*Use the Geocoder package or the csv file to add the columns **latitude** and **longitude** to the new dataframe*

In [198]:
!pip install geocoder

In [202]:
df['PostalCode']

0      M3A
1      M4A
2      M5A
3      M6A
4      M7A
      ... 
98     M8X
99     M4Y
100    M7Y
101    M8Y
102    M8Z
Name: PostalCode, Length: 103, dtype: object

In [217]:
# Let's give Geocoder a Try
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
latitude=[]
longitude=[]
tries=0

# loop until you get the coordinates
for eachPostalCode in df['PostalCode']:
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(eachPostalCode))
        lat_lng_coords = g.latlng
        print('.',end='')
        tries+=1
        if tries>100: break
    if tries>100:
        print('\nreally doesn\'t seem to work')
        break
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    print(eachPostalCode, 'done')

.....................................................................................................
really doesn't seem to work


We literally tried a hundred times, so let's add the csv file to the dataframe instead...

In [218]:
# wget alternative windows
import urllib.request

In [219]:
url = 'http://cocl.us/Geospatial_data'
filename = 'geoData.csv'
urllib.request.urlretrieve(url, filename)

('geoData.csv', <http.client.HTTPMessage at 0x1ff32ea7d88>)

In [225]:
# read geodata into dataframe
geoData=pd.read_csv('geoData.csv')

In [224]:
geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [230]:
# rename the column 'Postal Code' to 'PostalCode'
geoData.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geoData.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [243]:
# merge the 2 datasets in a way that only columns get added where we already have the postal code in a row
dfGeo=pd.merge(df,geoData, on='PostalCode')

In [244]:
dfGeo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [245]:
dfGeo.shape

(103, 5)

Nice! Seems to have worked :-)

## Part 3 - Exploring and Clustering 

*Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.*

*Just make sure:*

*to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.*

In [248]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>space_explorer</em>, as shown below.

In [252]:
address = 'Toronto, CA'

In [253]:
geolocator = Nominatim(user_agent="space_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.6534817, -79.3839347.


In [325]:
df=dfGeo

In [329]:
# create map of Toronto using latitude and longitude values of all neighbourhoods
map_folium = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_folium)  
    
map_folium

Let's look at the different boroughs

In [320]:
# get unique values of boroughs
uniqueBoroughs=df['Borough'].unique().tolist()
len(uniqueBoroughs)

10

We have 10 different boroughs

Let's create a map where the different boroughs are colored differently

In [387]:
# create map of Toronto using latitude and longitude values of all neighbourhoods
map_folium = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    if borough==uniqueBoroughs[0]: custCol='red' 
    if borough==uniqueBoroughs[1]: custCol='yellow' 
    if borough==uniqueBoroughs[2]: custCol='green' 
    if borough==uniqueBoroughs[3]: custCol='orange' 
    if borough==uniqueBoroughs[4]: custCol='blue' 
    if borough==uniqueBoroughs[5]: custCol='black' 
    if borough==uniqueBoroughs[6]: custCol='white' 
    if borough==uniqueBoroughs[7]: custCol='gray' 
    if borough==uniqueBoroughs[8]: custCol='pink' 
    if borough==uniqueBoroughs[9]: custCol='magenta' 
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=custCol,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_folium)  
    
map_folium

Let's look at the yellow one, which is ```uniqueBoroughs[1]```

In [334]:
uniqueBoroughs[1]

'Downtown Toronto'

That one seems to be called **Downtown Toronto**

In [383]:
# Filter for Downtown Toronto
filtered=df[['Borough']]==uniqueBoroughs[1]
df_downtown=df[filtered.values.tolist()]

In [384]:
# check size
df_downtown.shape

(19, 5)

In [386]:
# check filtered data
df_downtown.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


looks good

let's make a map of only this borough

In [392]:
# create map of Toronto using latitude and longitude values of all neighbourhoods
map_folium = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_folium)  
    
map_folium

Read Foursquare Credentials and Version **from local File**

In [279]:
cred=pd.read_json(r'cred_foursquare.json')

In [395]:
VERSION=cred['keys'][0]['scriptvalue'] # Foursquare API version
CLIENT_ID=cred['keys'][1]['scriptvalue'] # Foursquare ID
CLIENT_SECRET=cred['keys'][2]['scriptvalue'] # Foursquare Secret

LIMIT=100

Create a function to get nearby venues

In [393]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's try the funtion on our filtered dataframe

In [396]:
downtown_venues = getNearbyVenues(names=df_downtown['Neighborhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


let's have a look at the data

In [401]:
downtown_venues.shape

(1280, 7)

In [402]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [407]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst",16,16,16,16,16,16
Central Bay Street,77,77,77,77,77,77
Christie,19,19,19,19,19,19
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


### Let's cluster the neighborhoods

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [416]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()

In [411]:
downtown_grouped.shape

(19, 206)

#### Let's print each neighborhood along with the top 5 most common venues

In [417]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3                Café  0.04
4              Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1   Italian Restaurant  0.05
2         Burger Joint  0.04
3  Japanese Restaurant  0.04
4       Sandwich Place  0.04


----Christie----
           venue  freq
0  Grocery Store  0.21
1           Café  0.16
2           Park  0.11
3    Coffee Shop  0.11
4          Diner  0.05


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.06
1          Coffee Shop  0.06
2              Gay Bar  0.05
3     Sushi Restaurant  0.04
4           Restaurant  0.04


----Comm

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [418]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [450]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery,Café,Farmers Market,Park
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Harbor / Marina,Boutique
2,Central Bay Street,Coffee Shop,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Burger Joint,Sandwich Place,Café,Dessert Shop,Ice Cream Shop,Spa
3,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Nightclub,Gas Station,Diner,Restaurant,Athletics & Sports
4,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Gastropub,Hotel,Mediterranean Restaurant,Men's Store


Run *k*-means to cluster the neighborhood into 5 clusters.

In [442]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 2, 4, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [451]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Electronics Store,Hotel
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Yoga Studio,Park,Discount Store,Diner,Bar,Bank,Café,Beer Bar,Burger Joint


Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Great, that was fun!